In [ ]:
import os
import csv
from countries_languages import country_to_language
import pandas as pd
from difflib import get_close_matches
import numpy as np
import re
from transliterate import translit
from unidecode import unidecode
import Levenshtein
import requests
import bs4
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib.parse
from datetime import datetime
from functools import reduce
from urllib.parse import urlsplit

import wikipediaapi

import wikipedia

from urllib.parse import urlparse

import difflib

from datetime import datetime

import calendar

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import AssemblyHelpers
#from AssemblyHelpers import find_money_info_from_name


import requests
from bs4 import BeautifulSoup

def load_csv_dataset(file_path):
    return pd.read_csv(file_path)

countries_codes = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/countries_and_codes.csv')

#leagues value
leagues_value = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/Most Updated Edited Transfermarkt Dataset.csv')
#leagues value large 
leagues_value_large = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/most_updated_transfermarkt_dataset.csv')


In [ ]:
#BIG DATASET

large_df_combined = load_csv_dataset('updated COMBINED DF_WORKING.csv')


terrell_new_df = load_csv_dataset('CSVs of edge cases - for T/updated Terrell new DF.csv')
len(terrell_new_df), len(large_df_combined)


In [ ]:
#Group 1 

group_1_updated = load_csv_dataset('updated_Group1_fixes_complete_ii.csv')
len(group_1_updated)

In [ ]:
#Group 2 

online_lookup_group_2 = load_csv_dataset('Group 2 (for T) - Find their $ Info/Online lookup required - 1 name found.csv')

lookup_group_2 = load_csv_dataset('Group 2 (for T) - Find their $ Info/lookup_required_matches.csv')

cyrillic_group_2 = load_csv_dataset('Group 2 (for T) - Find their $ Info/Cyrillic_Found_Guys.csv')
len(online_lookup_group_2), len(lookup_group_2), len(cyrillic_group_2)

In [ ]:
#Group 3 

cyrillic_group_3 = load_csv_dataset('CSVs of edge cases - for F/Group 3 - Nobody Found/Cyrillic_Guys_Nobody_Found.csv')

zeros_group_3 = load_csv_dataset('CSVs of edge cases - for F/Group 3 - Nobody Found/odd_name_found_0_cases.csv')

online_lookup_group_3 = load_csv_dataset('CSVs of edge cases - for F/Group 3 - Nobody Found/Online lookup required - 0 names found.csv')

len(cyrillic_group_3), len(zeros_group_3), len(online_lookup_group_3)

In [ ]:
leagues_value

In [ ]:
# Other group

wrong_df = load_csv_dataset('CSVs of edge cases - for F/"other" group/updated Terrell lookup cases - wrong DF.csv')
wrong_df = wrong_df.drop(columns={'Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.2'})
lookup_required_dudes = load_csv_dataset('CSVs of edge cases - for F/"other" group/lookup_required_dudes.csv')

large_dataset_said_0 = load_csv_dataset('CSVs of edge cases - for F/"other" group/updated_large_dataset_said_0.csv')

match_date_filter_said_0_guys = load_csv_dataset('CSVs of edge cases - for F/"other" group/updated match date said zero guys.csv')

len(wrong_df), len(lookup_required_dudes), len(large_dataset_said_0), len(match_date_filter_said_0_guys)

## WORKING WITH WRONG DF

In [ ]:
from fuzzywuzzy import process

def find_close_matches_variable(this_jersey, dataset_nationality, threshold):
    """
    Find close matches of `this_jersey` in `dataset_nationality` using Levenshtein distance.

    Args:
    - this_jersey (str): The string to find close matches for.
    - dataset_nationality (list): List of strings to search for close matches in.
    - threshold (int): Minimum similarity score required for a match (default is 90).

    Returns:
    - List of strings from `dataset_nationality` that are close matches to `this_jersey`.
    """
    close_matches = process.extract(this_jersey, dataset_nationality, limit=None)
    return [match[0] for match in close_matches if match[1] >= threshold]

#threshold match one 
def threshold_player_match(this_jersey, dataset_nationality):
    THRESHOLD_NUM = 89
    #stops when it returns a name.
    #if it doesnt find a match keep lowering the threshold until you find a match
    #but if you get to threshold of like 50 first you stop and just abandon ship    

    # Loop until someone is found or threshold goes below 50
    while THRESHOLD_NUM >= 50:
        matches = find_close_matches_variable(this_jersey, dataset_nationality, THRESHOLD_NUM)
        if matches:
            #print(f"Player is {this_jersey}. Found matches: {matches}. threshold is {THRESHOLD_NUM}")
            return matches, THRESHOLD_NUM
            #break
        else:
            THRESHOLD_NUM -= 1

    # If threshold reaches below 50 without finding any matches
    if THRESHOLD_NUM < 50:
        return [f"No matches found even with the lowest threshold."], this_jersey #jersey was {this_jersey}


def filter_using_date_of_match(candidate_list_of_names_input, nationality_input, matchdate_Input, SALARY_BOOLEAN): 

    FILTERED_NAMES_USING_MATCH_DATE = AssemblyHelpers.multiNameMatchDateLookup(candidate_list_of_names_input, nationality_input, matchdate_Input) 

    #print('in fudom', type(FILTERED_NAMES_USING_MATCH_DATE))
    #1 MATCH
    if isinstance(FILTERED_NAMES_USING_MATCH_DATE, str) or isinstance(FILTERED_NAMES_USING_MATCH_DATE, np.str_): #YELLOW 3
        #print('were here', FILTERED_NAMES_USING_MATCH_DATE)
        #print('were in fudom in the right place')
        #NAME_FOR_SELENIUM_SEARCH = FILTERED_NAMES_USING_MATCH_DATE
        # MONEY_FOUND_ONLINE = useSeleniumToFindMoney(NAME_FOR_SELENIUM_SEARCH, nationality_input, matchdate_Input, SALARY_BOOLEAN)
        # print('made it out of using selenium')
        # money_thisplayer = [MONEY_FOUND_ONLINE, f'Online WAS Lookup Required - salary boolean is {SALARY_BOOLEAN}', True, False]
        money_thisplayer = [0, 'Lookup Required', True, False]
        return money_thisplayer, FILTERED_NAMES_USING_MATCH_DATE
    #MULTIPLE MATCHES
    elif(len(FILTERED_NAMES_USING_MATCH_DATE) >= 2):
        print(FILTERED_NAMES_USING_MATCH_DATE, type(FILTERED_NAMES_USING_MATCH_DATE), type(FILTERED_NAMES_USING_MATCH_DATE) == str)
        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT MULTIPLE RESULTS: {FILTERED_NAMES_USING_MATCH_DATE}', True, False]
        return money_thisplayer, FILTERED_NAMES_USING_MATCH_DATE
    #0 MATCHES
    elif(len(FILTERED_NAMES_USING_MATCH_DATE) == 0):
        money_thisplayer = [0, F'FILTERING USING MATCH DATE LEFT 0 RESULTS. BEFORE FILTERING, CANDIDATE NAMES WERE {candidate_list_of_names_input}', True, False]
        return money_thisplayer, ''
    

def check_tokens(this_jersey, match):
    jersey_tokens = this_jersey.lower().split()
    match_tokens = match.lower().split()
    
    if all(token in match_tokens for token in jersey_tokens):
        return True
    elif all(token in jersey_tokens for token in match_tokens) and len(jersey_tokens) == len(match_tokens) + 1:
        return True
    elif all(token in match_tokens for token in jersey_tokens) and len(jersey_tokens) > len(match_tokens):
        return True
    else:
        return False

def filter_multi_word_matches_by_jersey_tokens(jersey_tokens, potential_matches):
    filtered_matches = []

    for match in potential_matches:
        # Split each match into tokens
        match_tokens = match.split()

        # Count the number of tokens that start with each character in jersey_tokens
        match_token_start_chars = {token[0] for token in match_tokens}
        #print(match_token_start_chars, f"*{jersey_tokens}")


        #for token in jersey_tokens:
         #   print(token, token[0] in match_token_start_chars)

        # Check if the match contains at least one token for each character in jersey_tokens
        if all(token[0] in match_token_start_chars for token in jersey_tokens):
            filtered_matches.append(match)

    return filtered_matches

In [ ]:
wrong_df['New Found Name'] = ''
wrong_df['Status II'] = ''

In [ ]:
wrong_df[wrong_df['Status'] == 'SUCCESS']['Lookup Return Case'].value_counts()

In [246]:
success_status_counter = 0
for i in range(0, 1063):
    names_found = []
    num_identical_rows = 0
    num_names_this = 0
    index = i
    row = wrong_df.loc[index]
    # Extract relevant columns
    match = row['Match']
    date = row['Date']

    if type(row['Found Name']) == str:
        if row['Found Name'].startswith('['):
            FOUND_NAME = eval(row['Found Name']) 
        else:
            FOUND_NAME = row['Found Name']
    
    if row['Status II'] != 'Done':
    
        if row['Status'] == 'SUCCESS':
            #for all the success guys. 
            #these are all people that eventually T needs to use imputing methods.   
            NAMES_FOUND = row['Name(s) Found']

            ORIGINAL_JERSEY = row['ORIGINAL JERSEY']
            if ORIGINAL_JERSEY == 'evčík':
                ORIGINAL_JERSEY = 'P Ševčík'
            this_country_code = row['Team Country Code']
            if pd.isna(this_country_code):
                country_name = 'Namibia'
            else:
                country_name = countries_codes[countries_codes[' Code'] == this_country_code]['Country'].unique()[0]
            dataset_nationality = leagues_value[leagues_value['Team 1 Code'] == this_country_code]['Name'].unique()
            if 'Mohd' in ORIGINAL_JERSEY:
                ORIGINAL_JERSEY = ORIGINAL_JERSEY.replace('Mohd', 'Mohammed')
            if type(FOUND_NAME) == list:
                if NAMES_FOUND.startswith('['):
                    for name in eval(NAMES_FOUND):
                        FOUND_NAME.append(name)
                else:
                    FOUND_NAME.append(NAMES_FOUND)
                found_names = FOUND_NAME
            else:
                found_names = [FOUND_NAME, NAMES_FOUND]
            closest_name = filter_multi_word_matches_by_jersey_tokens([token[0] for token in ORIGINAL_JERSEY.split(' ')], found_names)

            threshold_last_name = threshold_player_match(ORIGINAL_JERSEY.split(' ')[-1], dataset_nationality)[0]
            if closest_name != []:
                intersection = [name for name in closest_name if name in found_names]
            else:
                intersection = [name for name in found_names if name in threshold_last_name]

            
            if list(set(intersection)) != []:
                if len(list(set(intersection))) == 1 and list(set(intersection)) != ['No matches found even with the lowest threshold.']:
                    if ORIGINAL_JERSEY == 'Mudir Al Radaei':
                        intersection = ['Mudir Abdurabu']
                    elif ORIGINAL_JERSEY == 'Noel-Mc Leod':
                        intersection = ['Kraig Noel-McLeod']
                    elif ORIGINAL_JERSEY == 'Nguyn Tin Duy':
                        intersection = ['Tien Duy Nguyen']
                    elif ORIGINAL_JERSEY == 'Abbas Al Hassan':
                        intersection = ['Abbas Al-Hassan']
                    elif ORIGINAL_JERSEY == 'Angel':
                        intersection = ['Wilker Ángel']

                    print(i, ORIGINAL_JERSEY, closest_name, found_names)
                    print(i, f'intersection of lists is says{list(set(intersection))}')

                    success_status_counter += 1
                    wrong_df.at[i, 'New Found Name'] = list(set(intersection))[0]
                    wrong_df.at[i, 'Status II'] = 'Done - New'
                else:
                    #16 cases here left
                    filtered_names_match_date = filter_using_date_of_match(list(set(intersection)), row['Nationality'], date, False)[1]
                    # if type(filtered_names_match_date) == str:
                    print(i, ORIGINAL_JERSEY, closest_name, found_names)
                    print(i, f'FILTERED intersection of lists is says{filtered_names_match_date}')
                    wrong_df.at[i, 'New Found Name'] = filtered_names_match_date#[name for name in closest_name if name in found_names][0]
                    wrong_df.at[i, 'Status II'] = 'Done - Match Date Filtered'
                    success_status_counter += 1
            else:
                0==0 #18 cases here
                success_status_counter += 1
                print(i, wrong_df.at[i, 'Date'], wrong_df.at[i, 'Nationality'], wrong_df.at[i, 'Match'], ORIGINAL_JERSEY, closest_name, found_names)
                print(i, f'lists is says{found_names}')
            
        else:
        
                      
            NAMES_FOUND = row['Name(s) Found']

            ORIGINAL_JERSEY = row['ORIGINAL JERSEY']
            if ORIGINAL_JERSEY == 'evčík':
                ORIGINAL_JERSEY = 'P Ševčík'
            this_country_code = row['Team Country Code']
            if pd.isna(this_country_code):
                country_name = 'Namibia'
            else:
                country_name = countries_codes[countries_codes[' Code'] == this_country_code]['Country'].unique()[0]
            dataset_nationality = leagues_value[leagues_value['Team 1 Code'] == this_country_code]['Name'].unique()
            if 'Mohd' in ORIGINAL_JERSEY:
                ORIGINAL_JERSEY = ORIGINAL_JERSEY.replace('Mohd', 'Mohammed')
            if type(FOUND_NAME) == list:
                if NAMES_FOUND.startswith('['):
                    for name in eval(NAMES_FOUND):
                        FOUND_NAME.append(name)
                else:
                    FOUND_NAME.append(NAMES_FOUND)
                found_names = FOUND_NAME
            else:
                found_names = [FOUND_NAME, NAMES_FOUND]
            closest_name = filter_multi_word_matches_by_jersey_tokens([token[0] for token in ORIGINAL_JERSEY.split(' ')], found_names)
     
            if len(closest_name) == 1:
                print(i, ORIGINAL_JERSEY, [name for name in closest_name if name in found_names]) #closest_name, found_names
                success_status_counter += 1
                
                #wrong_df['Status 2'] MAKE A NEW STATUS TO MARK THEM OFF ONCE DONE  
            else:
                print(i)
                
                threshold_last_name = threshold_player_match(ORIGINAL_JERSEY.split(' ')[-1], dataset_nationality)[0]
                if closest_name != []:
                    intersection = [name for name in closest_name if name in found_names and name in threshold_last_name]
                else:
                    intersection = [name for name in found_names if name in threshold_last_name]
                if len(list(set(intersection))) == 2:
                    
                    filtered_names_match_date = filter_using_date_of_match(list(set(intersection)), row['Nationality'], date, False)[1]
                    if type(filtered_names_match_date) == str:
                        print(i, ORIGINAL_JERSEY, closest_name, found_names)
                        print(i, f'SINGLE - filtered bymatch date - intersection of lists is says{filtered_names_match_date}')
                        wrong_df.at[i, 'New Found Name'] = filtered_names_match_date#[name for name in closest_name if name in found_names][0]
                        wrong_df.at[i, 'Status II'] = 'Done - Match Date Filtered'
                    
                        success_status_counter += 1
                    elif type(filtered_names_match_date) == list:
                        0==0
                        print(i, ORIGINAL_JERSEY, closest_name, found_names)
                        print(i, f'MULTI - filtered bymatch date - intersection of lists is says{filtered_names_match_date}')
                        wrong_df.at[i, 'New Found Name'] = filtered_names_match_date#[name for name in closest_name if name in found_names][0]
                        wrong_df.at[i, 'Status II'] = 'Done - Match Date Filtered'
                        success_status_counter += 1
                else:
                    filtered_names_match_date = filter_using_date_of_match(list(set(intersection)), row['Nationality'], date, False)[1]
                    print(i, ORIGINAL_JERSEY, closest_name, found_names)
                    print(i, f'FILTERED intersection of lists is says{filtered_names_match_date}')
                    success_status_counter += 1
                    #wrong_df.at[i, 'New Found Name'] = filtered_names_match_date#[name for name in closest_name if name in found_names][0]
                    #wrong_df.at[i, 'Status II'] = 'Done - Match Date Filtered'


                #print(i, f"threshold match says {threshold_player_match(ORIGINAL_JERSEY, found_names)}")
print(success_status_counter)

0


In [247]:


wrong_df[wrong_df['Status II'] == 'Done']#.tail(40)

,Match ID,Competition,Match,Date,Nationality,Team Country Code,Season,Status,Name,Market Value,Lookup Still Required?,Lookup Return Case,Impute Required?,Name(s) Found,ORIGINAL JERSEY,Match Case,Wrong Name,Found Name,New Found Name,Status II
0,3147,"World Cup Qualifiers, Europe, 2018",San Marino vs Czech Republic,26-03-2017,San Marino,SM,2017,SUCCESS,Davide Simoncini,0,False,Large Dataset had 0,False,Davide Simoncini,D Simoncini,single,Yes,['Aldo Simoncini'],Davide Simoncini,Done
1,3147,"World Cup Qualifiers, Europe, 2018",San Marino vs Czech Republic,26-03-2017,San Marino,SM,2017,SUCCESS,Fabio Vitaioli,0,False,Large Dataset had 0,False,Fabio Vitaioli,F Vitaioli,single,Yes,['Matteo Vitaioli'],Fabio Vitaioli,Done
2,2685,"World Cup Qualifiers, Asia, 2022",Maldives vs Guam,19-11-2019,Guam,GU,2019,SUCCESS,Shawn Nicklaw,0,False,Large Dataset had 0,False,Shawn Nicklaw,S Nicklaw,single,Yes,['Travis Nicklaw'],Shawn Nicklaw,Done
3,2685,"World Cup Qualifiers, Asia, 2022",Maldives vs Guam,19-11-2019,Guam,GU,2019,SUCCESS,Nate Lee,0,False,Large Dataset had 0,False,Nate Lee,N Lee,single,Yes,['Justin Lee'],Nate Lee,Done
4,2685,"World Cup Qualifiers, Asia, 2022",Maldives vs Guam,19-11-2019,Guam,GU,2019,SUCCESS,Jason Cunliffe,0,False,Large Dataset had 0,False,Jason Cunliffe,J Cunliffe,single,Yes,['A. J. DeLaGarza'],Jason Cunliffe,Done
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1058,3715,"Gold Cup, 2017",Costa Rica vs French Guyana,15-07-2017,Costa Rica,CR,2017,FAIL,Ariel Rodríguez,0,True,Was in DB before or after season,True,Ariel Rodríguez,A Rodríguez,single,Yes,"['Osvaldo Rodríguez', 'Francisco Rodríguez']",Ariel Rodríguez,Done
1059,2998,"World Cup Qualifiers, Europe, 2018",Georgia vs Serbia,24-03-2017,Serbia,RS,2017,FAIL,Boris Radunović,0,True,Was in DB before or after season,True,Boris Radunović,B Ivanović,single,Yes,"['Branislav Ivanovic', 'Djordje Ivanovic']",Branislav Ivanovic,Done
1060,3008,"World Cup Qualifiers, Europe, 2018",Serbia vs Wales,11-06-2017,Serbia,RS,2017,FAIL,Boris Radunović,0,True,Was in DB before or after season,True,Boris Radunović,B Ivanović,single,Yes,"['Branislav Ivanovic', 'Djordje Ivanovic']",Branislav Ivanovic,Done
1061,3024,"World Cup Qualifiers, Europe, 2018",Serbia vs Moldova,02-09-2017,Serbia,RS,2017,FAIL,Boris Radunović,0,True,Was in DB before or after season,True,Boris Radunović,B Ivanović,single,Yes,"['Branislav Ivanovic', 'Djordje Ivanovic']",Branislav Ivanovic,Done


In [248]:
wrong_df.to_csv('wrong_df_corrected.csv')

In [110]:


#do the threshold match. if its either Name(s) found column or Found Name column use that name 